In [172]:
import pandas as pd
import numpy as np
from scipy.stats import sem, t
from scipy import mean


In [177]:
#FUNCTION #1: Drop unneeded observations per the Paid Family Leave Policy Parameters

def abf_data(stateofwork, geography, selfemp, minsize, fed_gov, state_gov, local_gov, wagemax, benefits_taxed, statetax_amt, benefits, paytax_rate):
   
    #Restriction #1: Exclude those who are not in the formal economy. Drop COW=8 (Working without pay in family business or farm) & COW=9 (Unemployed and last worked 5 years ago or earlier or never)
    if ((d['COW'] == 8) | (d['COW'] == 9)).any():

        indexNames = d[ (d['COW'] == 8) | (d['COW'] == 9) ].index
        dropped_unemp =(len(indexNames))
        message_uemp = "We dropped %s unemployed workers from the dataset" % (dropped_unemp)
        print(message_uemp)
        d.drop(indexNames , inplace=True)
        
    else:
        print('Unemp workers are not in the dataset')
    
    #Restriction #2: Keep only selected State from either the "State or Work" or "State of Residence" ACS file
    if stateofwork==True:  #Use "ACS State of Work" File 
   
        if geography !=None: #Need to test if we get a Nationwide estimate if geography=True or None

            indexNames = d[d['POWSP'] != geography].index
            dropped_geog =(len(indexNames))
            message_geog = "We dropped %s workers from the dataset, who live outside the geographical bounds" % (dropped_geog)
            print(message_geog)
            d.drop(indexNames , inplace=True)
        else:
            print('State is null')

        
    elif not stateofwork:  #Use "ACS State of Residence" File  
    
        if geography !=None: #Need to test if we get a nationwide estimate if geography=True or None
                indexNames = d[d['ST'] != geography].index 
                dropped_geog =(len(indexNames))
                message_geog = "We dropped %s workers from the dataset, who live outside the geographical bounds" % (dropped_geog)
                print(message_geog)
                d.drop(indexNames , inplace=True)
        else:
            print('State is null')

    #Restriction #3: Exclude employers smaller than the minimum employer size parameter
    if minsize !=None: #None = Null Value

        emp_size_max_id = d['emp_size'].idxmax()
        emp_size_max_val=(d.loc[[emp_size_max_id], ['emp_size']]).values[0]
        
        if (emp_size_max_val > minsize):
            indexNames = d[d['emp_size'] < minsize].index #Need to confirm that the emp_size variable constructed on the microsim side wouldn't have any blanks
            type(indexNames)
            dropped_empsize =(len(indexNames))
            message_empsize = "We dropped %s workers from the dataset, at firms smaller than minsize" % (dropped_empsize)
            print(message_empsize)

            d.drop(indexNames , inplace=True)
        
        else:
            print('All employers are larger than the min size')
    else:
        print('There is no Minimum Emp Size restriction')

    #Restriction #4: Exclude those who are Self-Employed (drop self-employed: COW=6 & COW=7)
    if selfemp!=None: 
        
        if not selfemp: 
        
            if ((d['COW'] == 6) | (d['COW'] == 7)).any():
                indexNames = d[ (d['COW'] == 6) | (d['COW'] == 7) ].index
                dropped_selfemp =(len(indexNames))
                message_selfemp = "We dropped %s self-employed workers from the dataset" % (dropped_selfemp)
                print(message_selfemp)
                d.drop(indexNames , inplace=True)

            else:
                print('Self Emp workers are not in the dataset')
        else:
            print('Self-employed workers recieve PFML')
    
    #Restriction #5: Exclude federal gov workers, if policy applies (drop COW=5)
    if fed_gov!=None: #TRUE (Constraint is applied)
        
        if not fed_gov:
        
            if ((d['COW'] == 5)).any():
                indexNames = d[ (d['COW'] == 5)].index
                dropped_fed =(len(indexNames))
                message_fed = "We dropped %s federal employees from the dataset" % (dropped_fed)
                print(message_fed)
                d.drop(indexNames , inplace=True)
            else:
                print('Fed workers are not in the dataset')
        else:
            print('Fed employees recieve PFML')

    #Restriction #6: Exclude state govs workers, if policy applies (drop COW=4)
    if state_gov!=None: #TRUE (Constraint is applied)
       
        if not state_gov:
        
            if ((d['COW'] == 4)).any():
                indexNames = d[ (d['COW'] == 4)].index
                dropped_state =(len(indexNames))
                message_state = "We dropped %s state employees from the dataset" % (dropped_state)
                print (message_state)

                d.drop(indexNames , inplace=True)
            else:
                print('State workers are not in the dataset')
        else:
            print('State employees recieve PFML')
    
    #Restriction #7: Drop local gov workers, if policy applies (drop COW=3)
    if local_gov!=None: #TRUE (Constraint is applied)

        if not local_gov:
        
            if ((d['COW'] == 3)).any():
                indexNames = d[ (d['COW'] == 3)].index
                dropped_local =(len(indexNames))
                message_local = "We dropped %s local employees from the dataset" % (dropped_local) 
                print(message_local)
                d.drop(indexNames , inplace=True)
            else:    
                print('Local workers are not in the dataset')
        else:
            print('Local employees recieve PFML')
        
    #Apply Taxable Wage Max
    if wagemax!=None: #TRUE (Constraint is applied)
        d['income_final'] = np.where((d['income']>wagemax), wagemax, d['income'])                
        indexNames = d[ d['income']>wagemax].index
        censor=((len(indexNames)))
        message_censor = "We censored %s observations to the wage max" % (censor)
        print (message_censor)

#######################################################################################################
#OUTPUT DATASET
          
    #Append needed columns from data chunks into an aggregated dataframe
    policy2=['income_final', 'income_w', 'ptax_rev_w', 'ptax_rev_final', 'PWGTP', 'class', 'ST', 'POWSP', 'age_cat', "GENDER_CAT"]
    acs2=policy2+reps
    
    income_select= d.loc[:,acs2]
    global income_agg
    income_agg = pd.concat([income_agg,income_select])
    
    return


In [178]:
#FUNCTION #2: Conduct Final Calculations on the slimmer ABF Output dataset

def abf_calcs(stateofwork, geography, selfemp, minsize, fed_gov, state_gov, local_gov, wagemax, benefits_taxed, statetax_amt, benefits, paytax_rate):

    d=income_agg

    #########Step 1 - Calculate Point Estimates
    
    #Income
    #Intermediate output: unweighted income base (full geographic area)
    Total_income = d['income_final'].sum()

    #Total Weighted Income Base (full geographic area)
    d['income_w'] = d['income_final']*d['PWGTP']
    Total_income_w=d['income_w'].sum()
    print('Output: Weighted Income Base for Full Geographic Area:')
    print(Total_income_w)

    #Tax Revenue
    #Unweighted tax revenue collected (full geographic area)
    d['ptax_rev_final'] = d['income_final']*paytax_rate  
   
    #Total Weighted Tax Revenue (full geographic area)
    d['ptax_rev_w']=d['income_w']*paytax_rate
    Total_ptax_rev_w = paytax_rate*Total_income_w
    print('Output: Weighted Tax Revenue for Full Geographic Area:')
    print(Total_ptax_rev_w)

    message = "The weighted estimated tax revenue is %s based on a payroll tax rate of %s and a income base of %s " % (Total_ptax_rev_w, paytax_rate, Total_income_w)
    print(message)
    
    #State Tax Revenue Recouped from Taxed Benefits
    if benefits_taxed == True:
        recoup_tax_rev= statetax_amt* benefits
        print('Output: "State Tax Revenue Recouped from Taxed Benefits:')
        print(recoup_tax_rev)
        message = "With a state tax rate of %s and a benefit outlay of %s, the estimated state tax revenue is %s" % (statetax_amt, benefits, recoup_tax_rev)
        print(message)
    
#     #########Step 2 - Calculate Standard Errors with 80 Replicate Weights
    
    #Income
    income_r=[]
    for i in reps:
        income_r.append(((d['income_final']*d[i]).sum()))
    
    #print('80 Replicate Income:')
    #print(income_r)
    
    income_delta=[]
    for i in income_r:
        income_delta.append((i - Total_income_w)**2)
    
    income_se=((sum(income_delta))*(4/80))**.5

    
#     #Tax Revenue    
    tax_r=[]
    for i in reps:
        tax_r.append(((d['ptax_rev_final']*d[i]).sum()))
    
    #print('80 Replicate Tax Revenue:')
    #print(tax_r)
    
    tax_delta=[]
    for i in tax_r:
        tax_delta.append((i -Total_ptax_rev_w)**2)
    
    tax_se=((sum(tax_delta))*(4/80))**.5

    
#     ###########Step 3: Calculate Confidence Intervals
    
    #Income
    Total_Income_w_UCI = Total_income_w + 1.96*(income_se)
    print('Total Income Upper CI:')
    print(Total_Income_w_UCI)
    Total_Income_w_LCI = Total_income_w - 1.96*(income_se)
    print('Total Income Low CI:')
    print(Total_Income_w_LCI)
    
#     #Tax Revenue
    Total_ptax_w_UCI = Total_ptax_rev_w + 1.96*(tax_se)
    print('Total Income Upper CI:')
    print(Total_ptax_w_UCI)
    Total_ptax_w_LCI = Total_ptax_rev_w - 1.96*(tax_se)
    print('Total Income Low CI:')
    print(Total_ptax_w_LCI)
    
    
    #Return Dictionary with Final Output Values
    abf_output={'Total_income_w': Total_income_w, 'Total_income': Total_income,
                'income_se': income_se, 'Total_Income_w_UCI': Total_Income_w_UCI,'Total_Income_w_LCI': Total_Income_w_LCI, 
                'Total_ptax_rev_w': Total_ptax_rev_w,
                'tax_se': tax_se, 'Total_ptax_w_UCI': Total_ptax_w_UCI,'Total_ptax_w_LCI': Total_ptax_w_LCI,
                'recoup_tax_rev': recoup_tax_rev}     
    print(abf_output)
    
    pd.set_option('display.float_format', lambda x: '%.2f' % x) 
    print(pd.pivot_table(d,index=["class"],values=["income_w", "ptax_rev_w"],aggfunc=[np.sum]))
    print(pd.pivot_table(d,index=["age_cat"],values=["income_w", "ptax_rev_w"],aggfunc=[np.sum]))
    print(pd.pivot_table(d,index=["GENDER_CAT"],values=["income_w", "ptax_rev_w"],aggfunc=[np.sum]))
    
    
    return abf_output


In [181]:
#State of Residence
#df=pd.read_csv("ss16pnj.csv")
#df=pd.read_csv("ss16pri.csv")
#df=pd.read_csv("ss16pca.csv")

#State of Work
#df=pd.read_csv("xNJ_2012-2016p.csv")
#df=pd.read_csv("xRI_2012-2016p.csv")
#df=pd.read_csv("xCA_2012-2016p.csv")

#Load in Raw Data File
df2 = pd.DataFrame()
income_agg=pd.DataFrame()

#for chunk in pd.read_csv('xNJ_2012-2016p.csv', chunksize = 100000, low_memory=False):
for chunk in pd.read_csv('xRI_2012-2016p.csv', chunksize = 100000, low_memory=False):
#for chunk in pd.read_csv('xCA_2012-2016p.csv', chunksize = 100000, low_memory=False):
    df=pd.concat([df2,chunk])

#Select variables for analysis
    reps =['PWGTP'+str(i) for i in range(1,81)] 
    policy=["id","ST","POWSP", "emp_size", "COW", "ESR", "PERNP", "PWGTP", "WAGEP", "SEMP", "class", "AGEP", "SEX"]
    acs=policy+reps
    d= df.loc[:,acs]
    d['income']=d['PERNP']

#Create Class variable to aggregate the COW variable for display purposes
    d['class'] =d['COW']
    cleanup = {"class": {1: "Private", 2: "Private", 3:"Local", 4:"State", 5:"Federal", 6:"Self-Employed", 7:"Self-Employed", 8:"Other", 9:"Other"}}
    d.replace(cleanup, inplace=True)

#Create Age Categories for display purposes (need to find out variable specification on the microsim side)
    age_ranges = ["[{0} - {1})".format(AGEP, AGEP + 10) for AGEP in range(0, 100, 10)]
    age_ranges
    count_unique_age_ranges = len(age_ranges)
    count_unique_age_ranges
    d['age_cat'] = pd.cut(x=d['AGEP'], bins=count_unique_age_ranges, labels=age_ranges)

# Create Gender Categories for display pruposes
    d['GENDER_CAT']   = np.where(d['SEX']==1,'male','female')
    d['GENDER_CAT'] = np.where(np.isnan(d['SEX']), np.nan, d['GENDER_CAT']) #code missing responses as missing

    ######################
    #FUNCTION #1= abf_data: Drop unneeded observations per the Paid Family Leave Policy Parameters
    
    #CA Policy Parmeters
    #abf_data(stateofwork=True, geography=6, selfemp=False, minsize=None, fed_gov=False, state_gov=False, local_gov=False, wagemax=106742, benefits_taxed=True, statetax_amt=.15, benefits=5000000, paytax_rate=0.009)

    #New Jersey Policy Parmeters
    #abf_data(stateofwork=True, geography=34, selfemp=False, minsize=None, fed_gov=False, state_gov=True, local_gov=True, wagemax=32600, benefits_taxed=True, benefits_taxed=.15, benefits=5000000, paytax_rate=0.0008)
    #abf_data(stateofwork=True, geography=34, selfemp=False, minsize=None, fed_gov=False, state_gov=False, local_gov=False, wagemax=32600, benefits_taxed=True, statetax_amt=.15, benefits=5000000, paytax_rate=0.007)

    #Rhode Island Policy Parmeters 
    abf_data(stateofwork=True, geography=44, selfemp=False, minsize=None, fed_gov=False, state_gov=False, local_gov=False, wagemax=66300, benefits_taxed=True, statetax_amt=.15, benefits=175000000, paytax_rate=0.012)

#FUNCTION #2=abf_output: Conduct Final Calculations on the slimmer ABF Output dataset
#CA Policy Parmeters
#abf_output=abf_calcs(stateofwork=True, geography=6, selfemp=False, minsize=None, fed_gov=False, state_gov=False, local_gov=False, wagemax=106742, benefits_taxed=True, statetax_amt=.15, benefits=5000000, paytax_rate=0.009)

#NJ Policy Parmeters
#abf_output=abf_calcs(stateofwork=True, geography=34, selfemp=False, minsize=None, fed_gov=False, state_gov=True, local_gov=True, wagemax=32600, benefits_taxed=True, benefits_taxed=.15, benefits=5000000, paytax_rate=0.0008)
#abf_output=abf_calcs(stateofwork=True, geography=34, selfemp=False, minsize=None, fed_gov=False, state_gov=False, local_gov=False, wagemax=32600, benefits_taxed=True, statetax_amt=.15, benefits=5000000, paytax_rate=0.007)

#Rhode Island Policy Parmeters
abf_output=abf_calcs(stateofwork=True, geography=44, selfemp=False, minsize=None, fed_gov=False, state_gov=False, local_gov=False, wagemax=66300, benefits_taxed=True, statetax_amt=.15, benefits=175000000, paytax_rate=0.012)


C:\Users\jsmith\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


We dropped 475 unemployed workers from the dataset
We dropped 35972 workers from the dataset, who live outside the geographical bounds
There is no Minimum Emp Size restriction
We dropped 2142 self-employed workers from the dataset
We dropped 872 federal employees from the dataset
We dropped 1055 state employees from the dataset
We dropped 1522 local employees from the dataset
We censored 3963 observations to the wage max
Output: Weighted Income Base for Full Geographic Area:
13702949371.0
Output: Weighted Tax Revenue for Full Geographic Area:
164435392.452
The weighted estimated tax revenue is 164435392.452 based on a payroll tax rate of 0.012 and a income base of 13702949371.0 
Output: "State Tax Revenue Recouped from Taxed Benefits:
26250000.0
With a state tax rate of 0.15 and a benefit outlay of 175000000, the estimated state tax revenue is 26250000.0
Total Income Upper CI:
13920775060.804846
Total Income Low CI:
13485123681.195154
Total Income Upper CI:
167049300.72965816
Total Inc

(74160, 283)
